In [1]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(
                  os.path.dirname("test-mode2"), 
                  os.pardir)
)
sys.path.append(PROJECT_ROOT)

import pandas as pd
import numpy as np
import torch
import json
import random
import torch.nn as nn
import torch.nn.functional as F
from train import train, evaluate
from src.dataset import create_dataloader
from src.utils import feature_extraction_pipeline, read_features_files, choose_model, read_feature
from src.models.utils import SaveBestModel, weight_init
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from typing import Dict, Tuple, List
from sklearn.metrics import classification_report

# making sure the experiments are reproducible
seed = 2109
random.seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

def seed_worker(worker_id: int):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(seed)

In [2]:
features_path = "../features/propor2022/"

# loading training features
X_train = read_feature(path=features_path, fold="0", name="X_train.pth")
y_train = read_feature(path=features_path, fold="0", name="y_train.pth")
print(f"Train: {X_train.shape}, {y_train.shape}")

# loading validation features
X_valid = read_feature(path=features_path, fold="0", name="X_valid.pth")
y_valid = read_feature(path=features_path, fold="0", name="y_valid.pth")
print(f"Valid: {X_valid.shape}, {y_valid.shape}")

# loading testing features
X_test = read_feature(path=features_path, fold=None, name="X_test.pth")
y_test = read_feature(path=features_path, fold=None, name="y_test.pth")
print(f"Test: {X_test.shape}, {y_test.shape}")

Train: torch.Size([500, 1, 128000]), torch.Size([500, 3])
Valid: torch.Size([125, 1, 128000]), torch.Size([125, 3])
Test: torch.Size([308, 1, 128000]), torch.Size([308, 3])


In [3]:
# reading the parameters configuration file
params = json.load(open("../config/mode_1.json", "r"))

# parameters defination
k_fold = None
max_seconds = 16

if "kfold" in params.keys():
    k_fold = params["kfold"]["num_k"]

max_samples = max_seconds * int(params["sample_rate"])

if params["mode"] != "mode_3":
    feature_config = params["feature"]
else:
    feature_config = {}

feature_config["sample_rate"] = int(params["sample_rate"])
data_augmentation_config = params["data_augmentation"]
dataset = params["dataset"]
wavelet_config = params["wavelet"]
model_config = params["model"]
mode = params["mode"]

feat_path = os.path.join(params["output_path"], params["dataset"])

if dataset == "propor2022":
    if data_augmentation_config["target"] == "majority":
        data_augment_target = [0]
    elif data_augmentation_config["target"] == "minority":
        data_augment_target = [1, 2]
    elif data_augmentation_config["target"] == "all":
        data_augment_target = [0, 1, 2]
    else:
        raise ValueError("Invalid arguments for target. Should be 'all', 'majority' or 'minority")
else:
    raise NotImplementedError

In [4]:
# creating and defining the model
device = torch.device("cuda" if torch.cuda.is_available and model_config["use_gpu"] else "cpu")

model = choose_model(
    mode=mode,
    model_name=model_config["name"],
    device=device
)

optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=model_config["learning_rate"],
    betas=(0.9, 0.98),
    eps=1e-9
)
loss = torch.nn.CrossEntropyLoss()
scheduler = None

# creating the model checkpoint object
sbm = SaveBestModel(
    output_dir=os.path.join(model_config["output_path"], dataset, mode, model_config["name"]),
    model_name=model_config["name"]
)

# creating the training dataloader
training_dataloader = create_dataloader(
    X=X_train,
    y=y_train,
    feature_config=feature_config,
    wavelet_config=wavelet_config,
    data_augmentation_config=data_augmentation_config,
    num_workers=0,
    mode=mode,
    shuffle=False,
    training=True,
    batch_size=model_config["batch_size"],
    data_augment_target=data_augment_target,
    worker_init_fn=seed_worker,
    generator=g
)

# creating the validation dataloader
validation_dataloader = create_dataloader(
    X=X_valid,
    y=y_valid,
    feature_config=feature_config,
    wavelet_config=wavelet_config,
    data_augmentation_config=data_augmentation_config,
    num_workers=0,
    mode=mode,
    shuffle=False,
    training=False,
    batch_size=model_config["batch_size"],
    data_augment_target=data_augment_target,
    worker_init_fn=seed_worker,
    generator=g
)

train_f1, train_loss = train(
    device=device,
    dataloader=training_dataloader,
    optimizer=optimizer,
    model=model,
    loss=loss
)

valid_f1, valid_loss = evaluate(
    device=device,
    dataloader=validation_dataloader,
    model=model,
    loss=loss
)

# training loop
for epoch in range(1, model_config["epochs"] + 1):
    print(f"Epoch: {epoch}/{model_config['epochs']}")

    train_f1, train_loss = train(
        device=device,
        dataloader=training_dataloader,
        optimizer=optimizer,
        model=model,
        loss=loss
    )

    valid_f1, valid_loss = evaluate(
        device=device,
        dataloader=validation_dataloader,
        model=model,
        loss=loss
    )

    print(f"\nEpoch: {epoch}")
    print(f"Train F1-Score: {train_f1:1.6f}")
    print(f"Train Loss: {train_loss:1.6f}")
    print(f"Validation F1-Score: {valid_f1:1.6f}")
    print(f"Validation Loss: {valid_loss:1.6f}\n")

    # updating learning rate
    if not scheduler is None:
        scheduler.step()

Epoch: 1/50

Epoch: 1
Train F1-Score: 0.304867
Train Loss: 4.104018
Validation F1-Score: 0.294643
Validation Loss: 3.911951

Epoch: 2/50

Epoch: 2
Train F1-Score: 0.375542
Train Loss: 2.351047
Validation F1-Score: 0.277747
Validation Loss: 2.149970

Epoch: 3/50

Epoch: 3
Train F1-Score: 0.320854
Train Loss: 1.517057
Validation F1-Score: 0.294643
Validation Loss: 1.810435

Epoch: 4/50

Epoch: 4
Train F1-Score: 0.338026
Train Loss: 1.267802
Validation F1-Score: 0.325685
Validation Loss: 1.343883

Epoch: 5/50

Epoch: 5
Train F1-Score: 0.426446
Train Loss: 0.829553
Validation F1-Score: 0.294643
Validation Loss: 1.072516

Epoch: 6/50

Epoch: 6
Train F1-Score: 0.356739
Train Loss: 1.196708
Validation F1-Score: 0.294643
Validation Loss: 1.383997

Epoch: 7/50

Epoch: 7
Train F1-Score: 0.352692
Train Loss: 1.359238
Validation F1-Score: 0.294643
Validation Loss: 2.118066

Epoch: 8/50

Epoch: 8
Train F1-Score: 0.462913
Train Loss: 0.866303
Validation F1-Score: 0.294643
Validation Loss: 1.066356

